### Create resized dataset notebook

Description : 

Table of contents :

In [1]:
from cuttings import *
import random 

In [64]:
def dataset_resized(test_name,
                    label,
                     list_images,
                     date,
                     size=128,
                     train=True,
                     test=False,
                     folder_data="Cuttings_data/"):
    
    test_path = test_name+"/"
    
    # List all the images in the folder
    train_tiffs = os.listdir(folder_data+test_path)
    
    if train:
        name_csv = 'train_'+date+'.csv'
        path_cut = './train/cuttings_'+date+'/'
        path_csv = './train/csv/'
        
    if test:
        name_csv = 'test_'+date+'.csv'
        path_cut = "./test/cuttings_"+date+"/"
        path_csv = "./test/csv/"
    
    # create folders if not already existing
    try :
        os.listdir(path_cut)
    except FileNotFoundError : 
        os.mkdir(path_cut) 
        print("Directory '%s' created" %path_cut) 
    try :
        os.listdir(path_csv)
    except FileNotFoundError : 
        os.mkdir(path_csv) 
        print("Directory '%s' created" %path_csv) 
        # Need to create empty csv file with all the features
        df = pd.DataFrame(columns=['path',
                                   'image_name',
                                   'rock_type',
                                   'scan_name',
                                   'label_assigned',
                                   'area',
                                   'centroid',
                                   'bbox',
                                   'bbox_rect'])
        df.to_csv(path_csv+name_csv)
        
    # Import CSV
    df_metadata = pd.read_csv(path_csv+name_csv,index_col=0)
    
    WIDTH = size
    HEIGHT = size
    
    # Iterate over all the images 
    for image_i in list_images :
        
        # Extract the image number
        tiff = train_tiffs[image_i]
        
        cut = Cuttings(folder_data+test_path,tiff)
        
        image = cut.load_picture()
        
        mask = cut.assign_mask(image)
        
        dilated = cut.assign_label(mask)
        
        big_samples = cut.big_cuttings(dilated)
        
        label = label
        
        # Create dataframe to store the metadata for 1 scan
        df_scan = pd.DataFrame()
        
        # Create lists to store the metadata for 1 scan
        list_path = []
        list_image_name = []
        list_rock_type = []
        list_scan_name = []
        list_label_assigned = []
        list_area = []
        list_local_centroid = []
        list_bbox = []
        list_bbox_rect = []
        
        # Iterate on all the cuttings found on the pictures
        for sample_i in big_samples:
            
            # Extract the cuttings
            im_rect = img_as_ubyte(dilated == regionprops(dilated)[sample_i].label)
            contours,_ = cv2.findContours(im_rect, 1, 2)
            rect = cv2.minAreaRect(contours[0])
            if len(contours) >= 2:
                cmax = sorted(contours, key=cv2.contourArea, reverse=True)[0]
                rect = cv2.minAreaRect(cmax)
            box = cv2.boxPoints(rect)
            # Rotated rectangle bounding box coordinates
            box = np.int0(box)
            
            # Extract ractangular padded cutting
            cutting_to_save = cv2.resize(crop_rectangle(image*im_rect,box,rect),
                   (WIDTH,HEIGHT), 
                   interpolation=cv2.INTER_CUBIC)
            
            # Save cutting (original + mask)
            # Cutting
            cv2.imwrite(path_cut+test_name+'_'+train_tiffs[image_i][:-4]+'_'+str(sample_i)+'.png', cutting_to_save)

            # Save metadata : 
            # Path
            list_path.append(path_cut+test_name+'_'+train_tiffs[image_i][:-4]+'_'+str(sample_i)+'.png')
            # Image name 
            list_image_name.append(test_name+'_'+train_tiffs[image_i][:-4]+'_'+str(sample_i))
            # Rock type 
            list_rock_type.append(label)
            # Scan name 
            list_scan_name.append(train_tiffs[image_i][:-4])
            # Label_assigned
            list_label_assigned.append(sample_i)
            # Area
            list_area.append(regionprops(dilated)[sample_i].area)
            # Local Centroid
            list_local_centroid.append(regionprops(dilated)[sample_i].local_centroid)
            # Bbox
            minr, minc, maxr, maxc = regionprops(dilated)[sample_i].bbox
            list_bbox.append([minr, minc, maxr, maxc])
            # Bbox_rect
            minr_r, minc_r, maxr_r, maxc_r = list(box[0]),list(box[1]),list(box[2]),list(box[3])
            list_bbox_rect.append([ minr_r, minc_r, maxr_r, maxc_r])
            
        df_scan['path'] = list_path
        df_scan['image_name'] = list_image_name
        df_scan['rock_type'] = list_rock_type
        df_scan['scan_name'] = list_scan_name
        df_scan['label_assigned'] = list_label_assigned
        df_scan['area'] = list_area
        df_scan['centroid'] = list_local_centroid
        df_scan['bbox'] = list_bbox
        df_scan['bbox_rect'] = list_bbox_rect
        
        df_metadata = df_metadata.append(df_scan,ignore_index=True)
    df_metadata.to_csv(path_csv+name_csv)

In [30]:
df_config = pd.read_excel('config_file.xlsx')
df_config.head()

,test_name,label,train,test,list_start,list_end,list_step
0,ML-DB-Geo1,0,True,False,360,700,4
1,ML-DB-Geo2,0,True,False,120,600,4
2,ML-DB-Geo3,0,False,True,300,550,4
3,MS-DB-Geo01-1,1,True,False,200,900,10
4,MS-DB-Geo01-2,1,True,False,160,800,10


#### Dataset - 128x128

In [42]:
date = "resized_128"

In [43]:
for i in range(df_config.shape[0]):
    dataset_resized(df_config.loc[i,'test_name'],
                df_config.loc[i,'label'],
                np.arange(df_config.loc[i,'list_start'], df_config.loc[i,'list_end'], df_config.loc[i,'list_step']).tolist(),
                date,
                size=128,
                train=df_config.loc[i,'train'],
                test=df_config.loc[i,'test'],
                folder_data='/Users/nilso/Documents/EPFL/MA4/PDS Turberg/Cuttings_data/')

Directory './train/cuttings_resized_128/' created
Directory './train/csv_resized_128/' created
Directory './test/cuttings_resized_128/' created
Directory './test/csv_resized_128/' created


#### Number of rock per category :

In [60]:
df_train = pd.read_csv('train/csv_'+date+'/train_'+date+'.csv',index_col=0)
df_test = pd.read_csv('test/csv_'+date+'/test_'+date+'.csv',index_col=0)

# Train
print('Train')
for i in range(5):
    print('Number of rock type {} = {}'.format(i,df_train[df_train['rock_type'] == i].shape[0]))
print()
# Test
print('Test')
for i in range(5):
    print('Number of rock type {} = {}'.format(i,df_test[df_test['rock_type'] == i].shape[0]))

Train
Number of rock type 0 = 1846
Number of rock type 1 = 1033
Number of rock type 2 = 1660
Number of rock type 3 = 1497
Number of rock type 4 = 1374

Test
Number of rock type 0 = 709
Number of rock type 1 = 517
Number of rock type 2 = 559
Number of rock type 3 = 656
Number of rock type 4 = 398


#### Create uniform dataset

In [50]:
# Generate 1000 samples per rock type
list_im_train = []
random.seed(0)
for i in range(5):
    list_im_train+=random.sample(list(df_train[df_train['rock_type'] ==i].index.values),k = 1000)

# Save training/validation dataset
df_train.iloc[list_im_train].sort_index().reset_index(drop=True).to_csv('train/csv_'+date+'/train_'+date+'_final.csv')

# Generate 200 samples per rock type
list_im_test = []
for i in range(5):
    list_im_test+=random.sample(list(df_test[df_test['rock_type'] ==i].index.values),k = 200)

# Save test dataset
df_test.loc[list_im_test].sort_index().reset_index(drop=True).to_csv('test/csv_'+date+'/test_'+date+'_final.csv')

#### Compute the Mean and Std for standarisation

In [54]:
def MinMaxNormalization(image,vmin=0, vmax=1):
        arr = np.array(image).astype('float32')
        arr = (arr - arr.min()) / (arr.max() - arr.min())
        arr = (vmax - vmin) * arr + vmin
        return arr

In [61]:
df = pd.read_csv('train/csv_'+date+'/'+'train_'+date+'_final.csv',index_col=0)

vec_im = []
for im in df['path']:
    cut = Cuttings(im[:24],im[24:])
    image = cut.load_picture()
    image = MinMaxNormalization(image,vmin=0, vmax=1)
    vec_im += list(image.reshape(-1))
    
print('Mean :')
print(np.mean(vec_im))
print()
print('Std :')
print(np.std(vec_im))

Mean :
0.5157003

Std :
0.32948261
